## FISH Image Converter-local (version 1.0)

In [4]:
# Import packages
import os
import shutil
from smb.SMBConnection import SMBConnection
import tempfile
import re

import matplotlib.pyplot as plt
import numpy as np

# import tifffile
import bigfish
import bigfish.stack as stack
import bigfish.plot as plot
import bigfish.multistack as multistack
import bigfish.detection as detection

from fpdf import FPDF
import os
import glob
from datetime import datetime


#### Input Image info:

In [5]:
#input user info
your_name = ''
imaged_by = ''

#specify the location of your data on the NAS
input_directory = '' #/your/input/directory
output_directory = '' #/your/output/directory

#specify what you have in every channel
Cy5 = "" # your 670 channel (asymmetric control)
mCherry = "" # your 610 channel (query RNA)
FITC = "" # your protein channel (protein markers)
DAPI = "" 

# NAS credentials and connection information
nas_host = '' #your_nas_server_hostname_or_ip'
nas_user = '' #your_username
nas_password = '' #your_password
nas_share = '' #(specific for the O'Nishimura lab - edit accordingly)

# Define which channels to include in the composite image
include_channels = [Cy5, mCherry, FITC, DAPI]  # Modify this list to only include the channels you want


#### Connect to the NAS

In [6]:
experiment_ID = input_directory.split(os.path.sep)[-1]
output_directory = os.path.join(output_directory, experiment_ID, 'images')


# Connect to the NAS
conn = SMBConnection(nas_user, nas_password, 'python-client', nas_host, use_ntlm_v2=True)
conn.connect(nas_host, 445)

# Check if connection is successfully established
smb_directory_path = os.path.relpath(input_directory, f'/Volumes/{nas_share}')
shared_files = conn.listPath(nas_share, smb_directory_path) # move into folder with raw data
if shared_files:
    print("Connection is established.")
else:
    print("Connection is not established.")
    
# #print the filenames, save in a list 
# filenames = [shared_file.filename for shared_file in shared_files if shared_file.filename.endswith(("_R3D_D3D.dv","_R3D_REF.dv"))]
# filenames = sorted(filenames)
# filenames

Connection is established.


## <font color='red'>STOP!</font> Do not edit below this line

### 1. Code to read images directly from the NAS and save as a png

#### 1.1 List file paths and read image stacks

In [7]:
#List all the color images and brightfield images in separate lists
image_color_paths = []
bf_paths = []

for shared_file in shared_files:
    file_path = os.path.join(input_directory, shared_file.filename) 
    if shared_file.filename.endswith("_R3D_D3D.dv"):
        # If it does, append it to the image_colors_paths list
        image_color_paths.append(file_path)
        image_color_paths = sorted(image_color_paths)
    elif shared_file.filename.endswith("_R3D_REF.dv"):
        # If it ends with "_R3D_REF.dv", append it to the bf_paths list
        bf_paths.append(file_path)
        bf_paths = sorted(bf_paths)

#Read all the image_color stacks and extract the image identifiers to save in "subdirectories"
all_image_color_stacks = []
subdirectories = []
for image_color_path in image_color_paths:
    image_color_stack = stack.read_dv(image_color_path, sanity_check=True)
    all_image_color_stacks.append(image_color_stack)
    file_name = os.path.splitext(os.path.basename(image_color_path))[0]
    parts = file_name.split('_')
    numeric_part = None
    for i, part in enumerate(parts):
        if part == 'R3D':
            numeric_part = parts[i - 1]
            break
    if numeric_part is not None:
        subdirectories.append(numeric_part)

#Read all the bf stacks
all_bf_stacks = []
for bf_path in bf_paths:
    bf_stack = stack.read_dv(bf_path, sanity_check=True)
    all_bf_stacks.append(bf_stack)

#### 1.2 Split channels, max project and save in image subdirectory

In [8]:
# # #List all the color images and brightfield images in separate lists
# # Define which channels to include in the composite

def normalize(img):
    """Normalize image to [0, 1] range."""
    return (img - img.min()) / (img.max() - img.min()) if img.max() > 0 else img

# Iterate through all_image_color_stacks and extract channels
all_color_channel_list = []
for i, stack in enumerate(all_image_color_stacks):
    image_colors = all_image_color_stacks[i]
    
    # Create subdirectory for outputs
    output_subdirectory = os.path.join(output_directory, f"{subdirectories[i]}")
    os.makedirs(output_subdirectory, exist_ok=True)
    
    # Store extracted channels for composite
    channel_dict = {}

    # Extract and save individual channels
    for channel_index in range(image_colors.shape[0]):
        current_image = image_colors[channel_index, :, :]
        all_color_channel_list.append(current_image)

        # Dynamically assign channel names
        channel_name = None
        if channel_index == 0:
            channel_name = Cy5
        elif channel_index == 1:
            channel_name = mCherry
        elif channel_index == 2:
            channel_name = FITC
        elif channel_index == 3:
            channel_name = DAPI

        # Save each channel image (max projection in z-axis for 3D)
        max_projection = np.max(current_image, axis=0)  # Take max projection along z-axis
        plot_filename = os.path.join(output_subdirectory, f"{subdirectories[i]}_{channel_name}_deconvolved.png")
        plt.imsave(plot_filename, max_projection, cmap='gray')

        # Show the individual channel (for debugging/visualization)
        # plt.figure(figsize=(6, 6))
        # plt.imshow(max_projection, cmap='gray')
        # plt.title(f"{channel_name} Channel")
        # plt.axis('off')
        # plt.show()

        # Store channel for composite if selected
        if channel_name in include_channels:
            # Normalize and add to dictionary
            channel_dict[channel_name] = normalize(max_projection)  # Normalize the selected channels

    # Apply a scaling factor to increase visibility
    scale_factor = 2.0  # Adjust this value to see if it helps

    # Create composite image if selected channels exist
    if channel_dict:
        # Ensure all channels are the same size
        if len(set([img.shape for img in channel_dict.values()])) > 1:
            print(f"Warning: The images for {', '.join(channel_dict.keys())} have mismatched sizes.")
            continue  # Skip this stack if sizes don't match

        # Initialize the composite image (RGB, 3 channels)
        composite = np.zeros((channel_dict[next(iter(channel_dict))].shape[0], 
                             channel_dict[next(iter(channel_dict))].shape[1], 3))  # Initialize RGB image

        # Assign colors based on selected channels
        # Normalize and apply scaling factor to each channel before combining them
        if Cy5 in channel_dict:
            cy5_channel = normalize(channel_dict[Cy5]) * scale_factor  # Normalize and scale Cy5 channel
            composite[..., 0] = np.maximum(composite[..., 0], cy5_channel)  # Max red intensity for Cy5

        if mCherry in channel_dict:
            mcherry_channel = normalize(channel_dict[mCherry]) * scale_factor  # Normalize and scale mCherry channel
            composite[..., 0] = np.maximum(composite[..., 0], mcherry_channel)  # Max red intensity for mCherry
            composite[..., 2] = np.maximum(composite[..., 2], mcherry_channel)  # Max blue intensity for mCherry
            # This creates the magenta effect: red + blue

        if FITC in channel_dict:
            fitc_channel = normalize(channel_dict[FITC])  # Normalize FITC channel
            composite[..., 1] = np.maximum(composite[..., 1], fitc_channel)  # Max green intensity for FITC

        if DAPI in channel_dict:
            dapi_channel = normalize(channel_dict[DAPI])  # Normalize DAPI channel
            composite[..., 2] = np.maximum(composite[..., 2], dapi_channel)  # Max blue intensity for DAPI

        # Normalize the final composite image to the [0, 1] range
        composite = np.clip(composite / composite.max(), 0, 1) if composite.max() > 0 else composite

        # Apply brightness adjustment after composite is made (adjust to taste)
        brightness_factor = 2  # Adjust this value for brightness
        composite = np.clip(composite * brightness_factor, 0, 1)  # Apply brightness increase

        # Save the composite image
        composite_filename = os.path.join(output_subdirectory, f"{subdirectories[i]}_composite.png")
        plt.imsave(composite_filename, composite)

        # Show the composite image (for debugging/visualization)
        # plt.figure(figsize=(6, 6))
        # plt.imshow(composite)
        # plt.title("Composite Image")
        # plt.axis('off')
        # plt.show()

print(f"{len(subdirectories)} images saved as PNG in {output_directory}")


2 images saved as PNG in /Users/onishlab/Documents/01_bigfish/02_data/reports/231107_LP306_L4440/images


#### 1.3 Generate pdf report with deconvolved images (as .png)

In [9]:
# Generate report
# Define the desired order of the images on the pdf report
image_order = [Cy5, mCherry, FITC, DAPI, "composite"]

# Function to generate the cover letter
def add_cover_letter(experiment_details, readme_content):
    pdf.add_page()
    pdf.set_font("Arial", size=14)
    pdf.cell(100, 10, txt=(f"Experiment Title: {experiment_ID}"), ln=True, align='C')
    pdf.ln(2)

    # Add experiment details
    for detail in experiment_details:
        pdf.cell(200, 10, txt=detail, ln=True, align='L')
    
    pdf.ln(5)

    # Add README content
    pdf.multi_cell(0, 10, txt=readme_content)
    pdf.ln(10)

# Create a PDF report
pdf_filename = os.path.join(output_directory, "report.pdf")
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

# Find the file that ends with "README"
readme_file = None
for file_path in glob.glob(os.path.join(input_directory, '*README.*')):
    readme_file = file_path
    break

if readme_file:
    # Read README content
    with open(readme_file, "r") as file:
        readme_content = file.read()
else:
    readme_content = "No README file found."

# Add cover letter once at the beginning
today_date = datetime.now().strftime("%B %d, %Y")
add_cover_letter([f"Date of processing: {today_date}",
                  f"Imaged by: {imaged_by}",
                  f"Processed by: {your_name}",
                  f"channels: {image_order}",
                  f"channels in composite: {include_channels}",
                  " ",
                  "Readme file:"], readme_content)

# Add a single page for all images
pdf.add_page()
pdf.set_font("Arial", size=12)

# Iterate through subdirectories and add images
for subdirectory in sorted(subdirectories):
    # Add experiment title and image ID
    pdf.cell(200, 10, txt=f"Image ID: {experiment_ID}_{subdirectory}", ln=True, align='L')
    pdf.ln(5)

    # Get PNG files in the subdirectory
    subdirectory_path = os.path.join(output_directory, subdirectory)
    png_files = glob.glob(os.path.join(subdirectory_path, '*.png'))

    # Sort PNG files according to the desired image order
    png_files_sorted = sorted(png_files, key=lambda x: image_order.index(os.path.basename(x).split('_')[1]) if os.path.basename(x).split('_')[1] in image_order else len(image_order))

    # Calculate the number of columns that can fit on the page
    max_columns = 5  # You can adjust this value based on your preference
    num_columns = min(len(png_files_sorted), max_columns)

    # Calculate the width of each column
    column_width = 190 / num_columns

    # Add images to the PDF in columns with space between images
    for i, png_file in enumerate(png_files_sorted):
        x_position = pdf.get_x() + i % num_columns * column_width
        y_position = pdf.get_y() + int(i / num_columns) * 20  # Adjust space between rows

        pdf.image(png_file, x=x_position, y=y_position, w=column_width)
    
    pdf.ln(80 * ((len(png_files_sorted) - 1) // num_columns + 1))  # Adjust line height based on the number of rows

# Output the PDF file
pdf.output(pdf_filename)
print(f"PDF report created: {pdf_filename}")


PDF report created: /Users/onishlab/Documents/01_bigfish/02_data/reports/231107_LP306_L4440/images/report.pdf


### Code by Naly Torres. Last edited Jan. 30th, 2025